In [23]:
## non mi sembra funzionare l'ordinamento in base al recall !!! to be reviewed

In [1]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 9 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,3 months and 17 days !!!
H2O cluster name:,h2o_python
H2O cluster total nodes:,1
H2O cluster free memory:,22.55 Gb
H2O cluster total cores:,48
H2O cluster allowed cores:,48
H2O cluster status:,"locked, healthy"


In [3]:
DATAFILE = "/mnt/fs-h2o/Telco-Customer-Churn.csv"
data = h2o.import_file(DATAFILE, destination_frame= "xgb_data")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
data.head()

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,Yes
9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.1,1949.4,No
6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.8,3046.05,Yes
6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [5]:
data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [6]:
data['SeniorCitizen'] = data['SeniorCitizen'].asfactor()

In [7]:
# costrusco la lista delle features
features = ['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges']

In [8]:
train, valid = data.split_frame([0.85], seed=1234, destination_frames= ["xg_train",
                                                                              "xg_valid"])

In [9]:
# uso la grid search per ottimizzazione hyperparametri

In [10]:
ntrees_opt = [10, 20, 25, 30, 50]
max_depth_opt = [2,3,4,5,6,7]
learn_rate_opt = [0.01, 0.1, 0.2, 0.3, 0.5 ]

hyper_parameters = {"ntrees": ntrees_opt, "max_depth": max_depth_opt, "learn_rate": learn_rate_opt}

In [11]:
gs = H2OGridSearch(H2OXGBoostEstimator, hyper_params=hyper_parameters)

In [12]:
# è importante dare un id al modello, in modo da poterlo ritrovare facilmente in Flow
%time gs.train(x = features, y = 'Churn', training_frame = train, validation_frame = valid, model_id = "xgb_churn")

xgboost Grid Build progress: |████████████████████████████████████████████| 100%
CPU times: user 5.09 s, sys: 364 ms, total: 5.45 s
Wall time: 4min 24s


In [17]:
grid = gs.get_grid(sort_by="recall", decreasing=False)

In [18]:
grid

       learn_rate max_depth ntrees  \
0             0.1         2     10   
1             0.1         4     50   
2             0.5         3     10   
3            0.01         7     50   
4             0.2         7     25   
..  ..        ...       ...    ...   
145           0.2         2     20   
146           0.1         7     20   
147          0.01         5     10   
148          0.01         3     50   
149           0.1         2     50   

                                                            model_ids recall  
0      Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_2    1.0  
1    Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_132    1.0  
2     Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_10    1.0  
3    Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_146    1.0  
4     Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_88    1.0  
..                                                                ...

In [19]:
# addestro il modello usando i parametri identificati dal grid search (max_depth = 3, num_trees = 25)

In [ ]:
train, test = data.split_frame([0.9], seed=1234, destination_frames= ["xgb_train", "xgb_test"])

In [ ]:
model = H2OGradientBoostingEstimator(ntrees=25, max_depth=5, learn_rate = 0.5)

In [ ]:
# è importante dare un id al modello, in modo da poterlo ritrovare facilmente in Flow
%time model.train(x = features, y = 'Churn', training_frame = train, validation_frame = test, model_id = "gbm_churn")

In [ ]:
model

In [ ]:
model.model_performance(test)

In [20]:
# in questo modo possiamo scegliere uno dei modelli della griglia
best_gbm = grid.models[0]

In [22]:
best_gbm

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  Grid_XGBoost_xg_train_model_python_1579439308081_78827_model_2


Model Summary: 


,,number_of_trees
0,,10.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.16069373169242132
RMSE: 0.40086622667970084
LogLoss: 0.4991959313577736
Mean Per-Class Error: 0.24101756502707716
AUC: 0.8345162316448788
pr_auc: 0.5234181723668281
Gini: 0.6690324632897575

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4386869966983795: 


,,No,Yes,Error,Rate
0,No,3544.0,852.0,0.1938,(852.0/4396.0)
1,Yes,476.0,1113.0,0.2996,(476.0/1589.0)
2,Total,4020.0,1965.0,0.2219,(1328.0/5985.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.044423,0.566084,3.296201,3.296201,0.872340,0.566084,0.872340,0.566084,0.146429,0.146429,229.620061,229.620061
1,,2,0.110586,0.544803,2.537041,2.842002,0.671429,0.544806,0.752137,0.553354,0.167857,0.314286,153.704082,184.200244
2,,3,0.151229,0.500127,1.845349,2.574152,0.488372,0.519837,0.681250,0.544346,0.075000,0.389286,84.534884,157.415179
3,,4,0.273157,0.470511,1.933223,2.288062,0.511628,0.471792,0.605536,0.511960,0.235714,0.625000,93.322259,128.806228
4,,5,0.309074,0.450346,1.193233,2.160834,0.315789,0.454282,0.571865,0.505258,0.042857,0.667857,19.323308,116.083443
5,,6,0.418715,0.390019,1.172660,1.902080,0.310345,0.417160,0.503386,0.482189,0.128571,0.796429,17.266010,90.207997
6,,7,0.518904,0.356872,0.819879,1.693130,0.216981,0.367282,0.448087,0.460003,0.082143,0.878571,-18.012129,69.313037
7,,8,0.609641,0.278350,0.629762,1.534862,0.166667,0.304139,0.406202,0.436805,0.057143,0.935714,-37.023810,53.486157
8,,9,0.707940,0.234217,0.435989,1.382281,0.115385,0.245602,0.365821,0.410256,0.042857,0.978571,-56.401099,38.228114
9,,10,0.856333,0.208789,0.120337,1.163600,0.031847,0.214307,0.307947,0.376300,0.017857,0.996429,-87.966333,16.359981




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2020-01-19 16:18:50,0.484 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.734503,0.500000,0.693147,0.500000,0.000000,1.000000,0.735350
1,,2020-01-19 16:18:50,0.532 sec,1.0,0.481340,0.656440,0.787616,0.172208,2.044980,0.239599,0.480881,0.655555,0.802396,0.160729,2.129102,0.225898
2,,2020-01-19 16:18:50,0.563 sec,2.0,0.465668,0.626303,0.801277,0.214415,2.180221,0.270343,0.465243,0.625504,0.812817,0.205191,2.244834,0.272212
3,,2020-01-19 16:18:50,0.594 sec,3.0,0.452494,0.601183,0.801277,0.214415,2.180221,0.270343,0.451616,0.599570,0.812817,0.205191,2.244834,0.272212
4,,2020-01-19 16:18:50,0.626 sec,4.0,0.441451,0.580072,0.805431,0.215444,2.180221,0.270343,0.440663,0.578643,0.816205,0.206024,2.244834,0.272212
5,,2020-01-19 16:18:50,0.657 sec,5.0,0.432102,0.562024,0.822491,0.337154,2.642456,0.233250,0.431504,0.560923,0.828615,0.323100,2.573138,0.247637
6,,2020-01-19 16:18:50,0.739 sec,6.0,0.424157,0.546441,0.825167,0.355821,2.681842,0.235589,0.423545,0.545306,0.831677,0.340211,2.602460,0.237240
7,,2020-01-19 16:18:50,0.772 sec,7.0,0.417462,0.532977,0.825295,0.355956,2.681842,0.235255,0.416409,0.531063,0.831952,0.340495,2.602460,0.234405
8,,2020-01-19 16:18:50,0.804 sec,8.0,0.410644,0.519591,0.827095,0.364129,2.710869,0.241103,0.409483,0.517454,0.834025,0.347988,2.604151,0.234405
9,,2020-01-19 16:18:50,0.837 sec,9.0,0.405742,0.509305,0.830730,0.516596,3.084282,0.234586,0.404472,0.506962,0.839081,0.521856,3.296201,0.210775



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Contract.Month-to-month,3658.237549,1.000000,0.624922
1,InternetService.Fiberoptic,725.081360,0.198205,0.123863
2,tenure,646.953979,0.176849,0.110517
3,OnlineSecurity.No,583.715454,0.159562,0.099714
4,TechSupport.No,127.937492,0.034972,0.021855
5,MonthlyCharges,94.515320,0.025836,0.016146
6,Contract.Oneyear,17.469177,0.004775,0.002984
